In [1]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import cv2

# Model

In [2]:
class linear(nn.Module):
    def __init__(self, in_f, out_f):
        super(linear,self).__init__()
        self.l = nn.Linear(in_features=in_f, out_features=out_f)
        self.b = nn.BatchNorm1d(out_f)
        
    def forward(self, t):
        t = self.l(t)
        t= self.b(t)
        t = F.relu(t)
        return t

In [3]:
l = linear(1,2)
print(l)

linear(
  (l): Linear(in_features=1, out_features=2, bias=True)
  (b): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [4]:
def flatten(t):
    t = t.reshape(1,-1)
    t = t.squeeze()
    return t

In [5]:
# landmark

class Gen_landmark(nn.Module):
    def __init__(self):
        super(Gen_landmark,self).__init__()
        self.l1 = nn.Linear(in_features=132, out_features=128)
        self.l2 = nn.Linear(in_features=128, out_features=64)
        self.l3 = nn.Linear(in_features=64, out_features=32)
        self.l4 = nn.Linear(in_features=32, out_features=64)
        self.l5 = nn.Linear(in_features=64, out_features=128)
        self.l6 = nn.Linear(in_features=128, out_features=132)
        self.b = nn.BatchNorm1d(132)
    
    def forward(t):
        t = self.l1(t)
        t = self.l2(t)
        t = self.l3(t)
        ll = t
        t = self.l4(t)
        t = self.l5(t)
        t = self.l6(t)
        t = F.relu(self.b(t))
        return t, ll

In [6]:
ld = Gen_landmark()
ld

Gen_landmark(
  (l1): Linear(in_features=132, out_features=128, bias=True)
  (l2): Linear(in_features=128, out_features=64, bias=True)
  (l3): Linear(in_features=64, out_features=32, bias=True)
  (l4): Linear(in_features=32, out_features=64, bias=True)
  (l5): Linear(in_features=64, out_features=128, bias=True)
  (l6): Linear(in_features=128, out_features=132, bias=True)
  (b): BatchNorm1d(132, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [7]:
class conv_block(nn.Module):
    def __init__(self, in_f, out_f, s):
        super(conv_block, self).__init__()
        self.in_f, self.out_f, self.st = in_f,out_f, s
        self.conv = nn.Conv2d(in_channels=self.in_f, out_channels=self.out_f, kernel_size=5, stride = self.st, padding=2)
        self.batchNorm = nn.BatchNorm2d(self.out_f)
    
    def forward(self,t):
        t = self.conv(t)
        t = self.batchNorm(t)
        t = F.relu(t)
        return t

In [8]:
class deconv(nn.Module):
    def __init__(self, in_f, out_f, s):
        super(deconv, self).__init__()
        self.in_f, self.out_f ,self.st= in_f,out_f ,s
        self.deconv = nn.ConvTranspose2d(in_channels=self.in_f, out_channels=self.out_f, kernel_size=5, stride = self.st, padding=2, output_padding=1)
        self.batchNorm = nn.BatchNorm2d(self.out_f)
    
    def forward(self,t):
        t = self.deconv(t)
        t = self.batchNorm(t)
        t = F.relu(t)
        return t

In [9]:
conv = conv_block(1, 3, 2)
print(conv)

conv_block(
  (conv): Conv2d(1, 3, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (batchNorm): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [10]:
deconva = deconv(3,2,1)
print(deconva)

deconv(
  (deconv): ConvTranspose2d(3, 2, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), output_padding=(1, 1))
  (batchNorm): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [11]:
# image network

class generator(nn.Module):
    def __init__(self, ll):
        super(generator, self).__init__()
        self.ll = ll
        self.conv1 = conv_block(in_f=3, out_f= 64, s=2)
        self.conv2 = conv_block(in_f = 64, out_f=128, s=2)
        self.conv3 = conv_block(in_f = 128, out_f = 256, s=2)
        self.fc1 = nn.Linear(in_features=256*8*8, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=96)
        self.fc3 = nn.Linear(in_features=128, out_features=4096)
        self.deconv1 = deconv(256, 256, 2)
        self.deconv2 = deconv(in_f=256, out_f=128, s=2)
        self.deconv3 = deconv(in_f=128, out_f=64, s=2)
        self.deconv4 = deconv(in_f=128, out_f=64, s=2)
        self.out = conv_block(in_f=64, out_f=3, s=1)
    def forward(self, t):
        t = self.conv1(t)
        skip_t = t
        t = self.conv2(t)
        t = self.conv3(t)
        print(t.shape)
        t = flatten(t)
        t = self.fc1(t)
        t = self.fc2(t)
        t = torch.cat((t,self.ll), 0)
        print(t.shape)
        t =self.fc3(t)
        t = t.reshape(1,256,4,4)
        t = self.deconv1(t)
        print(t.shape)
        t = self.deconv2(t)
        print(t.shape)
        t = self.deconv3(t)
        print(t.shape)
        t = torch.cat((t,skip_t), 1)
        print(t.shape)
        t = self.deconv4(t)
        print(t.shape)
        t = self.out(t)
        print(t.shape)
        return t
        
        
    

In [12]:
ll = torch.randn(32)
gn = generator(ll)
gn

generator(
  (conv1): conv_block(
    (conv): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (batchNorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): conv_block(
    (conv): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (batchNorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): conv_block(
    (conv): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (batchNorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc1): Linear(in_features=16384, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=96, bias=True)
  (fc3): Linear(in_features=128, out_features=4096, bias=True)
  (deconv1): deconv(
    (deconv): ConvTranspose2d(256, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
    (batchNorm): BatchNorm2d(256, eps=1e-05, momentum=

In [13]:
r = torch.randn((1, 3, 64, 64))
r = gn.forward(r)
r.shape

torch.Size([1, 256, 8, 8])
torch.Size([128])
torch.Size([1, 256, 8, 8])
torch.Size([1, 128, 16, 16])
torch.Size([1, 64, 32, 32])
torch.Size([1, 128, 32, 32])
torch.Size([1, 64, 64, 64])
torch.Size([1, 3, 64, 64])


torch.Size([1, 3, 64, 64])

In [14]:
# discriminator
class LayerNormConv2d(nn.Module):
    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(features)).unsqueeze(-1).unsqueeze(-1)
        self.beta = nn.Parameter(torch.zeros(features)).unsqueeze(-1).unsqueeze(-1)
        self.eps = eps
        self.features = features

    def _check_input_dim(self, input):
        if input.size(1) != self.gamma.nelement():
            raise ValueError('got {}-feature tensor, expected {}'
                             .format(input.size(1), self.features))

    def forward(self, x):
        self._check_input_dim(x)
        x_flat = x.transpose(1,-1).contiguous().view((-1, x.size(1)))
        mean = x_flat.mean(0).unsqueeze(-1).unsqueeze(-1).expand_as(x)
        std = x_flat.std(0).unsqueeze(-1).unsqueeze(-1).expand_as(x)
        print(std.size())
        return self.gamma.expand_as(x) * (x - mean) / (std + self.eps) + self.beta.expand_as(x)

In [15]:
class disconv_block(nn.Module):
    def __init__(self,in_f,out_f,s):
        super(disconv_block, self).__init__()
        self.in_f, self.out_f, self.st = in_f,out_f, s
        self.conv = nn.Conv2d(in_channels=self.in_f, out_channels=self.out_f, kernel_size=5, stride = self.st, padding=2)
        self.LayerNorm = LayerNormConv2d(self.out_f)
        
    def forward(self,t):
        t = self.conv(t)
        t = self.LayerNorm(t)
        t = F.leaky_relu(t)
        return t

In [16]:
z = disconv_block(3,5,2)
c = torch.randn((1,3,32,32))
c = z(c)
c.shape

torch.Size([1, 5, 16, 16])


torch.Size([1, 5, 16, 16])

In [17]:
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.conv1 = disconv_block(in_f=3, out_f=64, s=2)
        self.conv2 = disconv_block(in_f=64, out_f=128, s=2)
        self.conv3 = disconv_block(in_f=128, out_f=256, s=2)
        self.conv4 = disconv_block(in_f=256, out_f=512, s=2)
        self.conv5 = disconv_block(in_f=512, out_f=1, s=2)
        
    def forward(self,t):
        t = self.conv1(t)
        t = self.conv2(t)
        t = self.conv3(t)
        t = self.conv4(t)
        t = self.conv5(t)
        return t

In [18]:
s = discriminator()
s

discriminator(
  (conv1): disconv_block(
    (conv): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (LayerNorm): LayerNormConv2d()
  )
  (conv2): disconv_block(
    (conv): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (LayerNorm): LayerNormConv2d()
  )
  (conv3): disconv_block(
    (conv): Conv2d(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (LayerNorm): LayerNormConv2d()
  )
  (conv4): disconv_block(
    (conv): Conv2d(256, 512, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (LayerNorm): LayerNormConv2d()
  )
  (conv5): disconv_block(
    (conv): Conv2d(512, 1, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (LayerNorm): LayerNormConv2d()
  )
)

In [19]:
x = torch.randn((1,3,64,64))
x = s(x)
x.shape

torch.Size([1, 64, 32, 32])
torch.Size([1, 128, 16, 16])
torch.Size([1, 256, 8, 8])
torch.Size([1, 512, 4, 4])
torch.Size([1, 1, 2, 2])


torch.Size([1, 1, 2, 2])

# Data Preprocessing

In [28]:
images = []
folder = "cohn-kanade-images/"
for person in os.listdir(folder):
    direc = os.path.join(folder,person)
    for exp in os.listdir(direc):
        dire = os.path.join(direc, exp)
        if(os.path.isdir(dire)):
            for emo in os.listdir(dire):
                img = cv2.imread(os.path.join(dire,emo))
            images.append(img)

In [29]:
len(images)

593

In [30]:
images[0].shape

(490, 640, 3)

# Losses

## Code Checking and Rough work

In [ ]:
x = nn.ConvTranspose2d(in_channels=2, out_channels=4, kernel_size=4, stride=2 )
y = torch.randn((1, 2, 4, 3))
b = nn.BatchNorm2d(4)
y = x(y)
y = b(y)

In [ ]:
print(y.shape)

In [ ]:
z = nn.Linear(100, 400)
y = torch.randn((1, 256, 4, 4))
decon = deconv(256, 256, 2)
y = decon(y)
print(y.shape)

In [ ]:
b = torch.randn(10)
c = nn.BatchNorm1d(10)
b = c(b)

In [ ]:
v = torch.tensor([[10,11,12]] , dtype = torch.float32)
b = nn.BatchNorm1d(1)
v = b(v)

In [ ]:
z = nn.Conv2d(in_channels=3, out_channels=10, stride=1, kernel_size=5, padding=2)
x = torch.randn((1,3,64,64))
x = z(x)

In [ ]:
x.shape

In [ ]:
x =z(x)
x.shape

In [ ]:
x =z(x)
x.shape

In [ ]:
t = torch.randn(4096)
t = t.reshape(1,256,4,4)

In [ ]:
t.shape

In [ ]:
z = nn.ConvTranspose2d(in_channels=256 ,out_channels=256, kernel_size=5, stride = 2, output_size=[1,256,8,8])
x = torch.randn((1,256,4,4)


In [ ]:
z = nn.ConvTranspose2d(in_channels=256 ,out_channels=256, kernel_size=5, stride = 2, padding=2, output_padding=1)
x = torch.randn((1,256,4,4))
x = z(x)
x.shape

In [ ]:
b = torch.randn(1,10,4,4)
a = torch.randn(1, 20, 4,4 )
t = torch.cat((a,b), 1)

In [ ]:
t.shape

In [ ]:
t = nn.LayerNorm(4)
x = torch.randn((1,4,16,4))
x = t(x)